In [166]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [178]:
botswana = pd.read_csv('botswana.tsv', sep = '\t')

Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

О каждой из них мы знаем:
- сколько детей она родила (признак ceb)

- возраст (age)

- длительность получения образования (educ)

- религиозная принадлежность (religion)

- идеальное, по её мнению, количество детей в семье (idlnchld)

- была ли она когда-нибудь замужем (evermarr)

- возраст первого замужества (agefm)

- длительность получения образования мужем (heduc)

- знает ли она о методах контрацепции (knowmeth)

- использует ли она методы контрацепции (usemeth)

- живёт ли она в городе (urban)

- есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

In [179]:
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [180]:
botswana.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
count,4361.000000,4361.000000,4361.000000,4241.000000,4354.000000,4290.000000,4361.000000,2079.000000,1956.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000
mean,2.441642,27.405182,5.855996,4.615892,0.963252,0.577622,0.476726,20.686388,5.144683,0.516625,0.140202,0.701766,0.092911,0.275815
std,2.406861,8.685233,3.927075,2.219303,0.188164,0.493996,0.499515,5.002383,4.803028,0.499781,0.347236,0.457535,0.290341,0.446975
min,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,20.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,1.000000,23.000000,8.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [181]:
botswana.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

In [185]:
botswana.shape[0]

4361

In [191]:
botswana_dropna = botswana.dropna(axis = 0)

for column in botswana_dropna.columns:
    print(column, ':', botswana_dropna[column].isnull().sum())
    
print(botswana_dropna.shape)

ceb : 0
age : 0
educ : 0
religion : 0
idlnchld : 0
knowmeth : 0
usemeth : 0
evermarr : 0
agefm : 0
heduc : 0
urban : 0
electric : 0
radio : 0
tv : 0
bicycle : 0
(1834, 15)


In [186]:
4361 - 1834

2527

In [140]:
botswana.insert(7, 'nevermarr', 0)

In [141]:
botswana.loc[botswana['agefm'].isnull(), 'nevermarr'] = 1

In [142]:
botswana.drop('evermarr', axis = 1, inplace = True)

In [143]:
botswana['agefm'].fillna(0, inplace = True)

In [144]:
botswana.loc[botswana['nevermarr'] == 1, 'heduc'] = -1

In [145]:
botswana['heduc'].isnull().sum()

123

In [146]:
botswana.insert(4, 'idlnchld_noans', 0)
botswana.loc[botswana['idlnchld'].isnull(), 'idlnchld_noans'] = 1
botswana['idlnchld'].fillna(-1, inplace = True)

In [150]:
botswana.insert(10, 'heduc_noans', 0)
botswana.loc[botswana['heduc'].isnull(), 'heduc_noans'] = 1
botswana['heduc'].fillna(-2, inplace = True)

In [152]:
botswana.insert(7, 'usemeth_noans', 0)
botswana.loc[botswana['usemeth'].isnull(), 'usemeth_noans'] = 1
botswana['usemeth'].fillna(-1, inplace = True)

In [161]:
botswana.dropna(axis = 0, inplace = True)

In [165]:
botswana.shape[0] * botswana.shape[1]

78264

In [167]:
botswana.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld_noans', 'idlnchld',
       'knowmeth', 'usemeth_noans', 'usemeth', 'nevermarr', 'agefm',
       'heduc_noans', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle'],
      dtype='object')

In [169]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld_noans + idlnchld + knowmeth + usemeth_noans + usemeth +'\
             'nevermarr + agefm + heduc_noans + heduc + urban + electric + radio + tv + bicycle', data = botswana)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                        23:30:42   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [170]:
print('Breusch-Pagan test: p = %f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p = 0.000000


In [171]:
m2 = smf.ols('ceb ~ age + educ + idlnchld_noans + idlnchld + knowmeth + usemeth_noans + usemeth +'\
             'nevermarr + agefm + heduc_noans + heduc + urban + electric + bicycle', data = botswana)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                        23:36:09   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [172]:
print("F = %f, p = %f, k1 = %f" % m1.fit().compare_f_test(m2.fit()))

F = 0.919236, p = 0.467231, k1 = 5.000000


In [173]:
m3 = smf.ols('ceb ~ age + educ + idlnchld_noans + idlnchld + knowmeth +'\
             'nevermarr + agefm + heduc_noans + heduc + urban + electric + bicycle', data = botswana)
fitted = m3.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                        23:39:57   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [175]:
print("F = %f, p = %f, k1 = %f" % m2.fit().compare_f_test(m3.fit()))
F, p, k1 = m2.fit().compare_f_test(m3.fit())
print(p)

F = 92.890582, p = 0.000000, k1 = 2.000000
3.1552009480386492e-40


In [177]:
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                        23:41:57   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.